In [ ]:
!pip install -qqq -U wandb --progress-bar off
import wandb
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

wb_token = userdata.get('wandb')
wandb.login(key=wb_token)

In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install datasets evaluate

In [ ]:
from datasets import load_dataset
mrqa = load_dataset("enriquesaou/mrqa-squadded-sample")

In [ ]:
mrqa

# Training

In [ ]:
# see https://huggingface.co/FacebookAI/roberta-base
base_model_id = "VMware/roberta-base-mrqa"#"FacebookAI/roberta-base"#

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained(base_model_id)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [ ]:
max_length = 512
stride = 128

In [ ]:
# source: https://github.com/huggingface/transformers/tree/main/examples/pytorch/question-answering
def preprocess_training(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer['text'][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
def preprocess_validation(examples):
        questions = [q.strip() for q in examples["question"]]
        inputs = tokenizer(
            questions,
            examples["context"],
            truncation="only_second",
            max_length=max_length,
            stride=stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        sample_mapping = inputs.pop("overflow_to_sample_mapping")

        inputs["example_id"] = []

        for i in range(len(inputs["input_ids"])):
            sequence_ids = inputs.sequence_ids(i)
            context_index = 1

            sample_index = sample_mapping[i]
            inputs["example_id"].append(examples["id"][sample_index])

            inputs["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(inputs["offset_mapping"][i])
            ]

        return inputs

In [ ]:
train_mrqa = mrqa['train'].map(
    preprocess_training,
    batched=True,
    remove_columns=mrqa['train'].column_names,
)

val_mrqa = mrqa['validation'].map(
    preprocess_validation,
    batched=True,
    remove_columns=mrqa['validation'].column_names,
)

mrqa, train_mrqa, val_mrqa

## Training loop

In [ ]:
my_model_id = "roberta-vmw-mrqa-waw"

In [ ]:
from transformers import TrainingArguments, Trainer

model.train()

training_args = TrainingArguments(
    output_dir=my_model_id,
    eval_strategy="epoch",
    learning_rate=3e-6,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="wandb",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_mrqa,
    eval_dataset=val_mrqa,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
"""
point = "checkpoint-1500"
model = AutoModelForQuestionAnswering.from_pretrained("roberta-vmw-mrqa-plus/"+point)
model.push_to_hub("roberta-vmw-mrqa-plus-"+point)
"""